<img width="8%" alt="TikTok.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/TikTok.png" style="border-radius: 15%">

# TikTok - Get liked videos by profile
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=TikTok+-+Get+liked+videos+by+profile:+Error+short+description">Bug report</a>

**Tags:** #tiktok #videos #snippet #content

**Author:** [Alex Nodeland](https://www.linkedin.com/in/alexnodeland/)

**Last update:** 2023-06-21 (Created: 2022-06-21)

**Description:** This notebook provides a script to retrieve a list of the liked videos of a given user on the popular social media platform, TikTok.

## Input

### Import libraries

In [1]:
import time
import os
import asyncio
import pandas as pd
import nest_asyncio
try:
    from openai import OpenAI
except:
    %pip install openai --user
    from openai import OpenAI
try:
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
except:
    %pip install selenium --user
    %pip install PyTikTokAPI --user
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
try:
    from bs4 import BeautifulSoup
except:
    %pip install beautifulsoup4 --user
    from bs4 import BeautifulSoup
try:
    from tiktokdl.download_post import get_post
except:
    %pip install tiktok-dlpy --user 
    %%python -m playwright install --with-deps
    from tiktokdl.download_post import get_post
nest_asyncio.apply()

### Setup variables

**Mandatory**
- `USERNAME`: The username of the TikTok user whose liked videos you want to retrieve.
- `OPENAI_API_KEY`: Your OpenAI API key.

**Optional**
- `LIMIT`: The maximum number of liked videos to retrieve. Default is None (no limit).

In [2]:
#Mandatory
USERNAME = '<username>'
OPENAI_API_KEY = '<openai_api_key>'

#Optional
LIMIT = 3

## Model

### Web automations

In [3]:
def click_liked_tab(driver):
    try:
        liked_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "p[data-e2e='liked-tab']"))
        )
        liked_tab.click()
        print("Clicked on Liked tab successfully")
    except Exception as e:
        print(f"Failed to click on Liked tab: {e}")

def scroll_to_load_all_videos(driver, max_scrolls=10):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolls = 0
    
    while scrolls < max_scrolls:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for page to load
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        scrolls += 1
        print(f"Scroll {scrolls}/{max_scrolls} completed")

### Scraping

In [4]:
def find_video_elements(soup):
    # Strategy 1: Look for the main container with data-e2e attribute
    main_container = soup.find('div', attrs={'data-e2e': 'user-liked-item-list'})
    if main_container:
        # Find all direct child divs that contain video items
        videos = main_container.find_all('div', attrs={'data-e2e': 'user-liked-item'}, recursive=False)
        if videos:
            return videos

    # Strategy 2: If the main container isn't found, search for video items directly
    videos = soup.find_all('div', attrs={'data-e2e': 'user-liked-item'})
    if videos:
        return videos

    # Strategy 3: Look for divs with specific class and role
    videos = soup.find_all('div', class_=lambda x: x and 'DivContainer' in x, attrs={'role': 'button', 'aria-label': 'Watch in full screen'})
    if videos:
        return videos

    # Strategy 4: Fallback to a more general approach
    videos = soup.find_all('div', class_=lambda x: x and ('ItemContainer' in x or 'VideoFeed' in x))
    return videos

def extract_video_info(video_element):
    try:
        video_link = video_element.find('a')['href']
        return {'url': video_link}
    except Exception as e:
        print(f"Error extracting video info: {e}")
        return None
    
async def download_tiktok_video(url, output_filename):
    try:
        video = await get_post(url)
        print(f"Downloading {url} to {output_filename}")
        if video.file_path:
            if os.path.abspath(video.file_path) != os.path.abspath(output_filename):
                os.rename(video.file_path, output_filename)
            print(f"Downloaded {url} to {output_filename}")
            return True
        else:
            print(f"Error: No file path found for {url}")
            return False
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return False

### Transcription

In [5]:
def transcribe_audio(file_path, client):
    try:
        with open(file_path, "rb") as audio_file:
            response = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        return response.text
    except Exception as e:
        print(f"Error transcribing {file_path}: {e}")
        return "Error"

### Script

In [6]:
async def main(username, limit=None):
    driver = webdriver.Chrome()
    
    # Navigate directly to the user's profile
    driver.get(f"https://www.tiktok.com/@{username}")
    print(f"Navigated to https://www.tiktok.com/@{username}")

    # Click on the Liked tab
    click_liked_tab(driver)

    # Scroll to load more videos
    scroll_to_load_all_videos(driver)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    videos = find_video_elements(soup)
    print(f"Found {len(videos)} videos")

    # Create a DataFrame to store video information
    video_data = []

    # Set up OpenAI client
    client = OpenAI(api_key=OPENAI_API_KEY)

    for video in videos:
        if limit and len(video_data) >= limit:
            break
        video_info = extract_video_info(video)
        if video_info:
            url = video_info['url']
            video_id = url.split('/')[-1]
            video_filename = f'{video_id}.mp4'
            
            try:
                # Download the TikTok video
                if await download_tiktok_video(url, video_filename):
                    # Transcribe the video
                    transcription = transcribe_audio(video_filename, client)
                    video_info['transcript'] = transcription
                else:
                    video_info['transcript'] = 'Error'
            finally:
                # Always try to remove the video file, even if an error occurred
                if os.path.exists(video_filename):
                    os.remove(video_filename)
            
            video_data.append(video_info)

    driver.quit()

    df = pd.DataFrame(video_data)
    df.to_csv(f'{username}_liked_videos_transcripts.csv', index=False, encoding='utf-8')
    print(f"Data saved to {username}_liked_videos_transcripts.csv")

    return df

## Output

### Get liked videos

In [7]:
liked_videos_transcripts = asyncio.run(main(USERNAME, LIMIT))
liked_videos_transcripts